In [19]:
import json
import pandas as pd
import numpy as np
from sklearn import metrics
from statistics import mode

In [2]:
df = pd.read_csv("all-li-qa.csv")
df.head()

,humanLoopName,text,passthrough_data,request_id,lacks_context,additional_context_website,claim_explanation,fact_checkable,fact_checkability,fact_check_website,...,client_token,request_id.1,account_id,account_name,text.1,external_response_id,severity,version,categories,latency
0,arn:aws:sagemaker:us-east-1:724151198517:flow-...,Covid is control 😎 #CovidIsOver Masks don’t wo...,"{""source"": ""twitter"", ""tweet_id"": 151869767591...",NaN,NaN,NaN,NaN,yes,5,https://www.usatoday.com/story/news/factcheck/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,arn:aws:sagemaker:us-east-1:724151198517:flow-...,Mike Lindell’s ‘Thanks-A-Thon’ Viewed by Dozen...,"{""request_id"": ""1-624e7f56-540bded2378d24f0513...","""1-624e7f56-540bded2378d24f05130d42f""",NaN,NaN,criticizing lindell over his ies and conspiracies,no,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,arn:aws:sagemaker:us-east-1:724151198517:flow-...,Same people who wanted Sputnik / RT gone for b...,"{""reddit_id"": ""t6f6js"", ""subreddit"": ""GenZedon...",NaN,NaN,NaN,opinion on the Ukraine crisis,no,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,arn:aws:sagemaker:us-east-1:724151198517:flow-...,Laut #handelsblatt sind Personalexperten inzwi...,{},NaN,NaN,NaN,NaN,no,1,NaN,...,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-6264dd7d-5d54ab13067e66027d56e174,NaN,NaN,Laut #handelsblatt sind Personalexperten inzwi...,8640fdbfff4847749f79df139529627f,1.0,1.0,NaN,5.207934
4,arn:aws:sagemaker:us-east-1:724151198517:flow-...,https://lnkd.in/g5vT4Urx\nPromesses électorale...,NaN,NaN,NaN,NaN,"Historic strike at CNES: ""Space projects stop ...",yes,1,https://www.francetvinfo.fr/sciences/espace/gr...,...,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-6260502d-5c683ccf38f3083b7375f1f4,NaN,NaN,https://lnkd.in/g5vT4Urx\nPromesses électorale...,8dd95b723c7347588f479df66ed4da19,1.0,1.0,NaN,1.820405


In [3]:
df.shape

(5721, 28)

In [11]:
sorted([name.replace("arn:aws:sagemaker:us-east-1:724151198517:flow-definition/", "") for name in df.humanLoopName.unique()])

['internal-misinfo-labeling-reddit-workflow-03222022',
 'midnight-high-qa-multi1-misinfo-labeling-04052022',
 'midnight-high-qa-multi1-misinfo-labeling-04122022',
 'midnight-high-qa-multi1-misinfo-labeling-04292022',
 'midnight-high-qa-multi2-misinfo-labeling-04052022',
 'midnight-high-qa-multi2-misinfo-labeling-04122022',
 'midnight-high-qa-single-misinfo-labeling-04052022',
 'midnight-high-qa-single-misinfo-labeling-04122022',
 'midnight-high-qa-single-misinfo-labeling-04292022',
 'midnight-high-vaco-all-misinfo-labeling-04082022',
 'midnight-high-vaco-core-misinfo-labeling-04212022',
 'midnight-high-vaco-training-misinfo-labeling-04212022',
 'midnight-misinfo-labeling-qa-multi1-workflow-03232022',
 'midnight-misinfo-labeling-qa-multi1-workflow-03312022',
 'midnight-misinfo-labeling-qa-multi2-workflow-03232022',
 'midnight-misinfo-labeling-qa-multi2-workflow-03312022',
 'midnight-misinfo-labeling-qa-single-workflow-03232022',
 'midnight-misinfo-labeling-qa-single-workflow-03312022',


In [12]:
df = df[df.humanLoopName.str.contains("midnight-normal-vaco-core-misinfo-labeling")]
df.shape

(1539, 29)

In [14]:
df.dubiousness.value_counts(normalize=True).sort_index()

1    0.818713
2    0.004548
3    0.103964
4    0.007147
5    0.065627
Name: dubiousness, dtype: float64

In [15]:
df.harmfulness.value_counts().sort_index()

1    1288
2       1
3     156
4      20
5      74
Name: harmfulness, dtype: int64

In [16]:
def get_severity(factcheck, harm, dubiousness):
    if factcheck >= 3:
        return 1
    elif harm >= 4:
        return 1
    elif dubiousness >= 4:
        return 1
    return 0
    
df["severity_vaco"] = df.apply(lambda row: get_severity(row["fact_checkability"], row["harmfulness"], row["dubiousness"]), axis=1)
df.severity_vaco.value_counts()

<ipython-input-16-7b654902a4e3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["severity_vaco"] = df.apply(lambda row: get_severity(row["fact_checkability"], row["harmfulness"], row["dubiousness"]), axis=1)


0    1269
1     270
Name: severity_vaco, dtype: int64

In [28]:
df.severity_vaco.isnull().sum()

0

In [17]:
df.text.duplicated().sum()

510

In [29]:
# what ratings do dup texts get?
for _, group in df.groupby("text"):
    if len(group) > 1:
        print(group.severity_vaco.tolist())

[0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0]
[0, 0]
[0, 0, 0, 0, 0, 0]
[1, 1]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0]
[1, 1]
[1, 1]
[1, 1, 1]
[1, 0]
[1, 1]
[1, 1]
[1, 1]
[0, 1]
[0, 0]
[1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0]
[0, 1]
[0, 0]
[1, 1, 1, 1]
[0, 0]
[0, 0]
[0, 1, 0]
[1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [32]:
# texts repeated five times
for _, group in df.groupby("text"):
    if len(group) > 5:
        print(group.iloc[0].text)
        print('\n-- TL --\n')

#Anxiety and #selfworth are often linked, if you feel #anxious about certain situations you are likely to feel less #confident about them too.

-- TL --

#stellefrei! Wer kennt geeignete Talente für diesen Job?

-- TL --

(English below) Ideen durchsetzen, Ziele erreichen: Sei ein Arschloch! - Wer seine Ideen durchsetzen und Ziele erreichen will, muss in gewisser Weise ein Arschloch sein. Zumindest gehört auch eine gewisse Portion Egoismus, dazu: Denn auch das nicht jedem schmeckt: Wer erfolgreich sein will, tut (meist) gut Mehr lesen unten!

 #BusinessIdeas #CareerManagement #Change #CorporateCulture #Creativity #Goal #Image #Innovation #JamesWatt #Psychology #Success #Money #Support #Climatechange #Investment #Education #Learn #Eco #Sustainable #Help #Information #Grateful

➖➖➖➖➖➖ABOUT➖➖➖➖➖➖➖➖
 Infos nach Wunsch: Top10-Blog, 500+ Bücher, Ratgeber-Magazine, Corporate Publishing & eLearning-on-Demand-Akademie. 20+ Jahre Erfahrung mit Partnern & Kunden wir dem Bundesbildungsministerium,

In [25]:
# take the most frequent rating
rows = []
for _, group in df.groupby("text"):
    row = group.iloc[0]
    if len(group) > 1:
        row['severity_vaco'] = mode(group.severity.tolist())
    rows.append(row)
    
df2 = pd.DataFrame(rows)
df2.shape

(1029, 29)

In [26]:
df2.text.duplicated().sum()

0

In [33]:
df2.severity.isnull().sum()

105

In [35]:
df2[df2.severity.isnull()].humanLoopName.value_counts()

arn:aws:sagemaker:us-east-1:724151198517:flow-definition/midnight-normal-vaco-core-misinfo-labeling-04182022    25
arn:aws:sagemaker:us-east-1:724151198517:flow-definition/midnight-normal-vaco-core-misinfo-labeling-04272022    22
arn:aws:sagemaker:us-east-1:724151198517:flow-definition/midnight-normal-vaco-core-misinfo-labeling-04262022    21
arn:aws:sagemaker:us-east-1:724151198517:flow-definition/midnight-normal-vaco-core-misinfo-labeling-04202022    20
arn:aws:sagemaker:us-east-1:724151198517:flow-definition/midnight-normal-vaco-core-misinfo-labeling-04192022    17
Name: humanLoopName, dtype: int64

### Historical P/R

In [36]:
subset = df2[df2.severity.notnull()]

for th in [3, 4, 5]:
    y_true = subset.severity_vaco
    y_pred = (subset.severity>=th).astype(int) # staging with translation feature
    print(
        f"threshold: {th}",
        "precision:", round(metrics.precision_score(y_true, y_pred), 3),
        "recall:", round(metrics.recall_score(y_true, y_pred), 3))
    print(metrics.confusion_matrix(y_true, y_pred))

threshold: 3 precision: 0.269 recall: 0.044
[[746  19]
 [152   7]]
threshold: 4 precision: 0.286 recall: 0.038
[[750  15]
 [153   6]]
threshold: 5 precision: 0.25 recall: 0.019
[[756   9]
 [156   3]]


## IGNORE code below

In [151]:
res_pkl = pd.read_pickle("responses_translate.pkl")
tmp = pd.DataFrame()
tmp["severity_staging"] = [item.json()["rating"]["severity"] if item.json().get("rating") else None for item in res_pkl]
tmp["logs_staging"] = [item.json() for item in res_pkl]
tmp["text"] = [item.json()['log_entry_json']['ratingRequest']['content']['text'] if item.json().get("log_entry_json") else None for item in res_pkl]
tmp = tmp[tmp.text.notnull()].drop_duplicates("text")

In [152]:
res.shape, tmp.shape

((871, 11), (840, 3))

In [153]:
res = res.merge(tmp, on="text", how="left")
res.shape

(871, 13)

In [155]:
res.severity_staging.isnull().sum()

31

In [156]:
subset = res[res.severity_staging.notnull()]

for th in [3, 4, 5]:
    y_true = subset.severity_vaco
    y_pred = (subset.severity>=th).astype(int) # last time run
    print(
        f"threshold: {th}",
        "precision:", round(metrics.precision_score(y_true, y_pred), 3),
        "recall:", round(metrics.recall_score(y_true, y_pred), 3))
    print(metrics.confusion_matrix(y_true, y_pred))

threshold: 3 precision: 0.227 recall: 0.056
[[733  17]
 [ 85   5]]
threshold: 4 precision: 0.235 recall: 0.044
[[737  13]
 [ 86   4]]
threshold: 5 precision: 0.222 recall: 0.022
[[743   7]
 [ 88   2]]


In [157]:
for th in [3, 4, 5]:
    y_true = subset.severity_vaco
    y_pred = (subset.severity_staging>=th).astype(int) # staging with translation feature
    print(
        f"threshold: {th}",
        "precision:", round(metrics.precision_score(y_true, y_pred), 3),
        "recall:", round(metrics.recall_score(y_true, y_pred), 3))
    print(metrics.confusion_matrix(y_true, y_pred))

threshold: 3 precision: 0.353 recall: 0.133
[[728  22]
 [ 78  12]]
threshold: 4 precision: 0.375 recall: 0.133
[[730  20]
 [ 78  12]]
threshold: 5 precision: 0.462 recall: 0.067
[[743   7]
 [ 84   6]]


In [168]:
subset.severity_vaco.value_counts(normalize=True)

0    0.892857
1    0.107143
Name: severity_vaco, dtype: float64

In [175]:
subset.shape

(840, 13)

In [176]:
subset = subset.merge(df[["text", "fact_checkability", "harmfulness", "dubiousness"]])

In [184]:
# false negative
for i, row in subset[((subset.severity_vaco==1) & (subset.severity_staging==1))].iterrows():
    print(row.content)
    print(row.severity_vaco, row.fact_checkability, row.harmfulness, row.dubiousness)
    print(row.severity, '->', row.severity_staging)
    print('---')

Carlos Sánchez
1 3 3 3
1 -> 1.0
---
Todos los días sirven para volver a empezar, buen día a todos.
1 3 3 3
1 -> 1.0
---
esto es España y todo vale  
1 3 3 3
1 -> 1.0
---
 Het grote pesten is begonnen in Ottawa 

Trudeau is dikke maatjes met z'n vriendje van de 'academie' in Davos, Macron. 

De foto's van innige omhelzingen hebben we de afgelopen dagen voorbij zien komen hier op Li.

Als een kat in het nauw die zich achter het gordijn heeft verstopt, moet je dan net zoals je buddy de uitweg zoeken in 'pesten':

-Dan kondig je de 'noodtoestand' af vanwege een vredelievend protest
-Dan stel je even een wet op, die het legaal maakt om mensen te arresteren die diesel of andere voorraden leveren aan truckers
-Dan ga je boetes uitschrijven aan de truckers

En wat nog meer? 

️✌️

#canada #gocanada #freedom
1 5 1 5
1 -> 1.0
---
ANGOLA - PORTUGAL - UK – IRLANDA – ESCÓCIA - EMIRATES - WORLD - - SOCIAL HOUSING DEVELOPMENT IN CAZENGA LUANDA JEOSAT ANGOLA - ÁLVARO MADALENO SOBRINHO – HÉLDER VIEIRA 

# Where are 500s?

In [161]:
res[res.severity_staging.notnull()].iloc[3].logs_staging

{'id': 'f3446cd8f038436ba7bd59506d0742c1',
 'rating': {'severity': 1, 'version': '1', 'categories': []},
 'log_entry_json': {'generatedRequestContext': {'awsTraceId': '1-627332c7-09622fea2813ff7972903037',
   'classifierConfig': [{'classifierName': 'MISINFORMATION',
     'isEnabled': True},
    {'classifierName': 'CONTROVERSIAL'},
    {'classifierName': 'OBJECTIONABLE'}],
   'experimentFlagConfig': {'enableGoogleSearchMisinfoModel': True,
    'enableTranslationToEnglish': True}},
  'misinfoServiceLog': {'claimReviewSearchResponse': {},
   'webSearchLogs': [{'loggableName': 'RAPID_API_1_UNLIMITED',
     'query': "HAEMATIDROSE ET HAEMOLACRIE UN CAS RAPPORT AU CONGRES ICD2021 Pendant le congrs #ICD2021, Dr Julio Cesar Salas Alanis a rapport le cas d'un patient de 22 ans qui, aprs un pisode motionnel pleurait de sang ou saignait par les yeux (Haemolacrie) et transpirait galement de sang (Haematidrose) sur une peau, les muqueuse et les organes taient intactes ou indemnes de lsions ou de dom

In [167]:
res.text.str.contains("Modernas co-founder created a quantum").sum()

0

In [164]:
res[res.severity_staging.isnull()].content.tolist()

['La Presidenta del Congreso de los Diputados ha destruido el mensaje del Presidente de Ucrania. Zelenski dice que PORCELANOSA, MAXAM y OTRAS empresas españolas siguen ganado dinero en Rusia. ¿NO OS DA VERGÜENZA??\n\nMi petición para que Zara abandonase Rusia: https://lnkd.in/eBwdkGMA\n\nPor si queréis escribirles:\n- Porcelanosa (Decoración) Atención al Cliente: https://lnkd.in/eBhaiDXs\n- Maxam (Armamento) Atención al Cliente: https://lnkd.in/eyNYiSAv\n\nYo ya lo he hecho.\n\nAlcampo, Leroy Merlin y Decathlon además de Danone y Nestlé siguen ganado dinero en Rusia.\n\n\nhttps://lnkd.in/eHpq3Aka',
 "Recherche active d’un stage de césure\n \n Cher Réseau,\n \n \u200dActuellement étudiante en Master 1 à Rennes School of Business, je suis à la recherche d’un\xa0#stage\xa0de 6 mois à partir de juillet prochain dans le domaine du\xa0#marketing\xa0:\n \n Localisation : France métropolitaine\n Disponibilité : À partir de juillet 2022\n Type de contrat : Stage de césure\n ✒️Service : Marketin

In [86]:
df.shape

(1517, 16)

In [50]:
for th in [3, 4, 5]:
    y_true = df.severity
    y_pred = (df.severity_staging>=th).astype(int)
    print(
        f"threshold: {th}",
        "precision:", round(metrics.precision_score(y_true, y_pred), 3),
        "recall:", round(metrics.recall_score(y_true, y_pred), 3))
    print(metrics.confusion_matrix(y_true, y_pred))

threshold: 3 precision: 0.5 recall: 0.231
[[68  6]
 [20  6]]
threshold: 4 precision: 0.545 recall: 0.231
[[69  5]
 [20  6]]
threshold: 5 precision: 0.5 recall: 0.154
[[70  4]
 [22  4]]


In [51]:
for th in [3, 4, 5]:
    y_true = df.severity
    y_pred = (df.severity_new>=th).astype(int)
    print(
        f"threshold: {th}",
        "precision:", round(metrics.precision_score(y_true, y_pred), 3),
        "recall:", round(metrics.recall_score(y_true, y_pred), 3))
    print(metrics.confusion_matrix(y_true, y_pred))

threshold: 3 precision: 0.5 recall: 0.231
[[68  6]
 [20  6]]
threshold: 4 precision: 0.556 recall: 0.192
[[70  4]
 [21  5]]
threshold: 5 precision: 0.5 recall: 0.154
[[70  4]
 [22  4]]


In [40]:
counts = df.severity.value_counts().sort_index()
pcts = df.severity.value_counts(normalize=True).sort_index().round(3)
pd.concat([counts, pcts], axis=1)

,severity,severity
1,89009,0.955
3,840,0.009
4,1123,0.012
5,2185,0.023


In [41]:
df = df.iloc[:100]

In [42]:
res = pd.read_pickle("responses_from_LI.pkl")
df["severity_new"] = [item.json()["rating"]["severity"] for item in res]
df["logs"] = [item.json() for item in res]
df["language"] = [item.json()["log_entry_json"]['processedQuery']["language"] for item in res]
df.head()

,timestamp,client_token,request_id,content,external_response_id,severity,version,categories,latency,severity_new,logs,language
0,2022-04-24 10:53:14.966972,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-62652c17-7154ab0c71b1f5cc4bf49bae,Die Nachbarn bauen\nTag 1,f41a93f34245444fb3d5edfd5c2e65ee,1,1,NaN,3.021534,1,"{'id': 'd38e025548dd41d7897d34cae5f3ddc1', 'ra...",de
1,2022-04-24 10:53:12.964059,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-62652c16-65a9b9854515a3b42697ca49,"Die Impfquote ist in Norwegen übrigens 74,5%. ...",21b681f167ed4f9ca50822d56e3ce07b,1,1,NaN,2.519114,1,"{'id': '48f6d9b787c04f7f82399d4fdba15f5c', 'ra...",de
2,2022-04-24 10:53:12.460512,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-62652c14-64eb28ed53f972f9223df06d,PETITION / Demande de REFERENDUM pour l'ABOLIT...,c84e9b756b5647d2bf434dfc6e045174,1,1,NaN,3.528190,1,"{'id': '0f12732ba0af44f594d07ccc1da90e5d', 'ra...",fr
3,2022-04-24 10:53:11.997844,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-62652c13-24cf769c38aeb46a22d4f3a0,Voor de afdeling Technisch Beheer is Linthorst...,c51e8b00e0394dacb159fdeb5e12df41,1,1,NaN,4.560903,1,"{'id': '4bb0accb6ab64b5e9729ab5be132a51f', 'ra...",nl
4,2022-04-24 10:53:11.893265,78bec1da-1e7a-41f9-8646-93e6e58ec31e,1-62652c10-38a2aec60879cccd7c0b16a3,"Extra, Extra! Indulto a los asintomáticos!!\n\...",1c28c4b8abb84109a8f0ea57ada4d8c3,1,1,NaN,7.441665,4,"{'id': 'aabe2be23e844f17b870334a5b1113cb', 'ra...",es


In [43]:
df.language.value_counts()

fr    24
en    22
de    20
nl    14
es    13
pt     6
lb     1
Name: language, dtype: int64

In [44]:
df[["severity", "severity_new"]].drop_duplicates()

,severity,severity_new
0,1,1
4,1,4
36,4,1
78,5,5
81,5,1


In [45]:
df["changed"] = df.severity != df.severity_new
df["change"] = df.apply(lambda row: f"{row.severity}->{row.severity_new}", axis=1)
df.change.value_counts()

1->1    95
1->4     2
4->1     1
5->5     1
5->1     1
Name: change, dtype: int64

In [46]:
df.changed.value_counts()

False    96
True      4
Name: changed, dtype: int64

In [54]:
df[df.change=="1->4"].iloc[1].content

'Deutschland ist in Gefahr, die Besten in Massen zu verlieren: Einer repräsentativen INSA-Umfrage zufolge denkt jede/r Sechste in Deutschland ernsthaft ans Auswandern; bei den gut Ausgebildeten und noch lange Berufstätigen ist es jede/r Vierte. Insgesamt denken nur 2/3 über diese Frage nicht nach. Ein Drittel der Bevölkerung überlegt, auszuwandern, jeder sechste ernsthaft! Das hat es seit der Nazizeit nicht mehr gegeben, es ist ein riesiges Alarmzeichen an eine Regierung: Mehr als 13 (in Worten: dreizehn!) Millionen Menschen denken ERNSTHAFT darüber nach, auszuwandern. Man möchte jede/n einzelne/ Abgeordnete/n in Berlin wachrütteln: "Hallo! Habt Ihr den Schuss IMMER NOCH NICHT gehört?!" Aber Regierung und Opposition - mit Ausnahme ausgerechnet der AfD - schunkelt gerade unisono dem Abgrund entgegen.... (Quelle: reitschuster.de)'

In [51]:
df[df.change=="5->1"].iloc[0].logs

{'id': '6ff1dafefa8d4135b3832ea14d06b5ff',
 'rating': {'severity': 1, 'version': '1', 'categories': []},
 'log_entry_json': {'generatedRequestContext': {'awsTraceId': '1-6271e230-4b6e2a793f0077225d72f572',
   'classifierConfig': [{'classifierName': 'MISINFORMATION',
     'isEnabled': True},
    {'classifierName': 'CONTROVERSIAL'},
    {'classifierName': 'OBJECTIONABLE'}],
   'experimentFlagConfig': {'enableGoogleSearchMisinfoModel': True,
    'enableTranslationToEnglish': True}},
  'misinfoServiceLog': {'claimReviewSearchResponse': {},
   'webSearchLogs': [{'loggableName': 'RAPID_API_1_UNLIMITED',
     'query': 'At all times be aware that information manipulation is a powerful tool Putins regime will not hesitate to use in achieving its goals. Poynter Institute offer tips&tricks to evaluate objectivity of information on social media. https://lnkd.in/gzyPcReX',
     'results': [{'annotations': {},
       'description': "At all times be aware that information manipulation is a powerful t

In [46]:
# y_true = df.label

# for name in ["translated", "no translation"]:        
#     print()
#     print(name)
#     for th in [3, 4, 5]:
#         if name == 'translated':
#             y_pred = (df.rating_translate>=th).astype(int)
#         else:
#             y_pred = (df.rating_no>=th).astype(int)
#         print(
#             f"threshold: {th}",
#             "precision:", round(metrics.precision_score(y_true, y_pred), 3),
#             "recall:", round(metrics.recall_score(y_true, y_pred), 3))
#         print(metrics.confusion_matrix(y_true, y_pred)
# )


translated
threshold: 3 precision: 0.295 recall: 0.684
[[449  31]
 [  6  13]]
threshold: 4 precision: 0.226 recall: 0.368
[[456  24]
 [ 12   7]]
threshold: 5 precision: 0.3 recall: 0.316
[[466  14]
 [ 13   6]]

no translation
threshold: 3 precision: 0.262 recall: 0.579
[[449  31]
 [  8  11]]
threshold: 4 precision: 0.2 recall: 0.316
[[456  24]
 [ 13   6]]
threshold: 5 precision: 0.25 recall: 0.263
[[465  15]
 [ 14   5]]


In [48]:
df[df.rating_no!=df.rating_translate].language.value_counts()

de    2
pt    2
nl    2
en    2
Name: language, dtype: int64

In [49]:
df[df.rating_no!=df.rating_translate].sort_values("language")

,tl_internal_id,language,label,text,rating_translate,json_translate,json_no,rating_no
50,502467423,de,0,"#Klima: #Dürre bedroht Millionen - ""Am Rande d...",2,<Response [200]>,<Response [200]>,5
351,1090082939,de,1,Die Impfungen sind für die Amerikanischen Sold...,3,<Response [200]>,<Response [200]>,1
285,514220213,en,0,Si parte! Allacciatevi le cinture! Sarà un via...,4,<Response [200]>,<Response [200]>,1
387,2827386453,en,0,People Starts Crying Badly In Movie Theater Af...,1,<Response [200]>,<Response [200]>,2
84,2891481729,nl,1,Dijsselbloem geeft namens de OVV een schoolvoo...,2,<Response [200]>,<Response [200]>,1
216,44528007,nl,1,"Okay, dit is toch wel heel zorgelijk aan het w...",5,<Response [200]>,<Response [200]>,1
64,1307301715,pt,0,#AgirÉurgente Artistas e organizações da socie...,2,<Response [200]>,<Response [200]>,1
454,4068785419,pt,0,"Desde sábado (26), o presidente ucraniano tem ...",2,<Response [200]>,<Response [200]>,1


In [50]:
df.iloc[387].json_translate.json()

{'id': 'f23c5f643ae14cf3b72f9912390d9934',
 'rating': {'severity': 1, 'version': '1', 'categories': []},
 'log_entry_json': {'generatedRequestContext': {'classifierConfig': [{'classifierName': 'MISINFORMATION',
     'isEnabled': True},
    {'classifierName': 'CONTROVERSIAL'},
    {'classifierName': 'OBJECTIONABLE'}],
   'experimentFlagConfig': {'enableClassifierV2': True,
    'enableTranslationToEnglish': True}},
  'misinfoServiceLog': {'claimReviewSearchResponse': {}},
  'ratingRequest': {'account': {'id': 'johnsmith', 'name': 'John Smith'},
   'content': {'text': 'People Starts Crying Badly In Movie Theater After Watching The Kashmir Files\xa0\n\n#TheKashmirFiles\nAmazing Cast:\n#AnupamKher\n#MithunChakraborty\n#DarshanKumaar\n#PallaviJoshi\n#ChinmayMandlekar\n#PrakashBelawadi\n#PuneetIssar\n#BhashaSumbli\n#SouravVerma\n#MrinalKulkarni\n#AtulSrivastava\n#AmaanIqbal'},
   'context': {'clientToken': '200b76c9-bc02-4397-a2d0-1440c9e1f6a0'}},
  'ratingResponse': {'id': 'f23c5f643ae14cf3b

In [51]:
df.iloc[387].json_no.json()

{'id': 'adabe21ea177420f93d09fd5717f002c',
 'rating': {'severity': 1, 'version': '1', 'categories': []},
 'log_entry_json': {'generatedRequestContext': {'classifierConfig': [{'classifierName': 'MISINFORMATION',
     'isEnabled': True},
    {'classifierName': 'CONTROVERSIAL'},
    {'classifierName': 'OBJECTIONABLE'}],
   'experimentFlagConfig': {'enableClassifierV2': True}},
  'misinfoServiceLog': {'claimReviewSearchResponse': {}},
  'ratingRequest': {'account': {'id': 'johnsmith', 'name': 'John Smith'},
   'content': {'text': 'Kirsty Allsopp told me to cancel my Netflix subscription in order to buy a house. \n\nWhat a joke… \n\nHere’s how many years of cancelled Netflix it would take to afford the average house price 👇🏻\n\nCc: Nathan Bickerton on Twitter \n#houseprice #whatajoke'},
   'context': {'clientToken': '200b76c9-bc02-4397-a2d0-1440c9e1f6a0'}},
  'ratingResponse': {'id': 'adabe21ea177420f93d09fd5717f002c',
   'severity': 1,
   'version': '1'}}}

## Test rating quality

In [146]:
with open("raw_data_strat.json", "r") as f:
    data = json.loads(f.read())

In [147]:
for k, v in data.items():
    print(k, len(v))

https://staging.api.trustlab.com/rating/single/v1 250
http://k8s-testxiao-midnight-3c42a8dbfe-afb6e1fc76d6412a.elb.us-east-1.amazonaws.com/rating/single/v1 250


In [148]:
test_data = data["http://k8s-testxiao-midnight-3c42a8dbfe-afb6e1fc76d6412a.elb.us-east-1.amazonaws.com/rating/single/v1"]
staging_data = data["https://staging.api.trustlab.com/rating/single/v1"]

In [197]:
tmp = pd.DataFrame()
tmp["text"] = [item['log_entry_json']['ratingRequest']['content']['text'] for item in test_data]
tmp["test_rating"] = [item['rating']['severity'] for item in test_data]
tmp["language"] = [item['log_entry_json']['processedQuery']['language'] for item in test_data]
tmp["translated_text"] = [item['log_entry_json']['processedQuery']['translatedText'] for item in test_data]
tmp["test_json"] = test_data

tmp2 = pd.DataFrame()
tmp2["text"] = [item['log_entry_json']['ratingRequest']['content']['text'] for item in staging_data]
tmp2["staging_rating"] = [item['rating']['severity'] for item in staging_data]
tmp2["staging_json"] = staging_data

tmp = tmp.merge(tmp2, on="text")
tmp.head()

,text,test_rating,language,translated_text,test_json,staging_rating,staging_json
0,"#PriyankaOnABP has stolen the show. But, she h...",1,en,"#PriyankaOnABP has stolen the show. But, she h...","{'id': 'cb0522ddf36245c3905777a51ec42603', 'ra...",1,"{'id': '3efc1728d5124249938518d778998e05', 'ra..."
1,Les informations que des troupes de l'ONU sont...,1,fr,Reports that UN troops are on Canadian soil an...,"{'id': '95167e50e1e34f2bb018244b1e09f87a', 'ra...",1,"{'id': '4a7e95ad0767483c8d8ea328eb0968a2', 'ra..."
2,"Athletes Dropping Dead of Cardiac Arrest for ""...",5,en,"Athletes Dropping Dead of Cardiac Arrest for ""...","{'id': '5bfcd39db04a4f81a87b77f63e70293e', 'ra...",5,"{'id': 'aa4702cd58bc43caad4099a7ce374be3', 'ra..."
3,I’ve been watching the local news and keeping ...,1,en,Ive been watching the local news and keeping u...,"{'id': '20f9f67f50764bd59a9d64168494fdec', 'ra...",1,"{'id': '4346ac82ede247599b28eb960e418ee7', 'ra..."
4,"« 700.000 €, pour une personne qui assure la c...",1,fr,"700,000 for a person who provides operational ...","{'id': '8be0e7e42c1e40e29383119904753df1', 'ra...",1,"{'id': 'b3f75c1f9ecb4d57b8759c97d50e1b8d', 'ra..."


In [198]:
tmp["test_misinfo"] = (tmp.test_rating>=3).astype(int)
tmp["staging_misinfo"] = (tmp.staging_rating>=3).astype(int)
tmp.head()

,text,test_rating,language,translated_text,test_json,staging_rating,staging_json,test_misinfo,staging_misinfo
0,"#PriyankaOnABP has stolen the show. But, she h...",1,en,"#PriyankaOnABP has stolen the show. But, she h...","{'id': 'cb0522ddf36245c3905777a51ec42603', 'ra...",1,"{'id': '3efc1728d5124249938518d778998e05', 'ra...",0,0
1,Les informations que des troupes de l'ONU sont...,1,fr,Reports that UN troops are on Canadian soil an...,"{'id': '95167e50e1e34f2bb018244b1e09f87a', 'ra...",1,"{'id': '4a7e95ad0767483c8d8ea328eb0968a2', 'ra...",0,0
2,"Athletes Dropping Dead of Cardiac Arrest for ""...",5,en,"Athletes Dropping Dead of Cardiac Arrest for ""...","{'id': '5bfcd39db04a4f81a87b77f63e70293e', 'ra...",5,"{'id': 'aa4702cd58bc43caad4099a7ce374be3', 'ra...",1,1
3,I’ve been watching the local news and keeping ...,1,en,Ive been watching the local news and keeping u...,"{'id': '20f9f67f50764bd59a9d64168494fdec', 'ra...",1,"{'id': '4346ac82ede247599b28eb960e418ee7', 'ra...",0,0
4,"« 700.000 €, pour une personne qui assure la c...",1,fr,"700,000 for a person who provides operational ...","{'id': '8be0e7e42c1e40e29383119904753df1', 'ra...",1,"{'id': 'b3f75c1f9ecb4d57b8759c97d50e1b8d', 'ra...",0,0


In [199]:
sum(tmp.test_rating!=tmp.staging_rating)

15

In [200]:
sum(tmp.test_misinfo!=tmp.staging_misinfo)

9

In [201]:
df_strat = pd.read_csv("[Goldilocks] POC 2 Base Samples - [DO NOT MODIFY] stratified_500 2022-04-01.csv")
df_strat.head()

,tl_internal_id,language,label,text
0,3629820281,en,MISINFORMATION,As Pfizer announce profits of nearly $40bn fro...
1,3693315959,en,CLEAR,Notice how every C-19 mandate has been the per...
2,2797061223,en,CLEAR,Are you kidding me? \n\nJUST IN: Guantanamo de...
3,3649246237,fr,MISINFORMATION,DECES & BLESSES GRAVES APRES LES INJECTIONS EX...
4,1644454476,en,MISINFORMATION,"Jump to conclusions, go ahead \n\nhttps://lnkd..."


In [202]:
tmp = tmp.merge(df_strat[["text", "label"]], on="text", how="left")
tmp["label"] = (tmp.label=="MISINFORMATION").astype(int)

In [228]:
tmp[tmp.language!="en"][["test_strat", "staging_strat"]]

,test_strat,staging_strat
1,FN,FN
4,TN,TN
6,TN,TN
8,TN,TN
11,FN,FN
13,TP,TP
15,TN,TN
18,TN,TN
20,FP,TN
21,TN,TN


In [231]:
tmp[tmp.test_misinfo!=tmp.staging_misinfo]

,text,test_rating,language,translated_text,test_json,staging_rating,staging_json,test_misinfo,staging_misinfo,label,test_type,staging_type,test_strat,staging_strat
20,https://lnkd.in/gmV3A4Pn\n\nDe geschiedenis he...,3,nl,https://lnkd.in/gmV3A4Pn History repeats itsel...,"{'id': '999ea0559755494082cfc209d7e70378', 'ra...",1,"{'id': '234bbf586abb4963928c4419f7cd5426', 'ra...",1,0,0,FP,TN,FP,TN
27,Rappel pour mes Amis Canadiens : M. Trudeau a ...,5,fr,Reminder for my Canadian Friends Mr. Trudeau f...,"{'id': 'f357f4b1a0934d0d94554de6c6d70b9a', 'ra...",1,"{'id': 'b46f7d44b327488c8375b63b10279ea3', 'ra...",1,0,1,TP,FN,TP,FN
91,Le Pr Montagnier est le premier à avoir parlé ...,4,fr,Professor Montagnier is the first to have spok...,"{'id': '7a2f9a1ed3c641dc85d26e77e70af11e', 'ra...",1,"{'id': '3dabab6db728463b9557cedce4d86c29', 'ra...",1,0,1,TP,FN,TP,FN
97,"Guerre en Ukraine : d'après les espions, Pouti...",5,fr,"War in Ukraine According to spies, Putin is su...","{'id': '380127d11c19452480f9e87ca2e20a51', 'ra...",1,"{'id': '3829fa8db9dc4d11add25f5330eed6d4', 'ra...",1,0,1,TP,FN,TP,FN
98,"STOP OU ENCORE ?\n\nLe passe sanitaire, nous d...",5,fr,"STOP OR AGAIN The health pass, we are told, ha...","{'id': 'e8873203128a459fbc5a0b8c70a176a9', 'ra...",2,"{'id': '38645743aa514a2eb6c811d2568ad6f8', 'ra...",1,0,1,TP,FN,TP,FN
184,Kritisch artikel over hoe de CDC omgegaan is m...,1,nl,Critical article on how the CDC has handled th...,"{'id': '7f1a30a398c447d784ea0c02369e84a8', 'ra...",3,"{'id': 'c29bde90fc704eb581872455333df941', 'ra...",0,1,1,FN,TP,FN,TP
210,Erbeutete Geheimdokumente: Ukraine forschte an...,4,de,Secret documents captured: Ukraine researching...,"{'id': 'af86475d239e4243b7281d7c9401dfc8', 'ra...",2,"{'id': '5db6f123e5d2460a865866f252a099aa', 'ra...",1,0,1,TP,FN,TP,FN
243,VINDT U DIT NOG STEEDS OKAY ??????\n\nDE UPDAT...,5,nl,"DO YOU STILL FIND THIS OKAY THE FEBRUARY 5, 20...","{'id': '00e1deadfb5f4598931cc893975b6e2c', 'ra...",1,"{'id': '1c826b2408bf485fa44e0fd515c32545', 'ra...",1,0,1,TP,FN,TP,FN
249,OTRA VIDA ROTA TRAS LA VACUNA\n\nEs peor el re...,5,es,ANOTHER BROKEN LIFE AFTER THE VACCINE The cure...,"{'id': '8bb80157be40411489a2c17bb2e8c644', 'ra...",1,"{'id': 'b26a648205234b20935fcfb65fbe5a22', 'ra...",1,0,1,TP,FN,TP,FN


In [230]:
tmp[tmp.test_misinfo!=tmp.staging_misinfo].translated_text.tolist()

['https://lnkd.in/gmV3A4Pn History repeats itself. But compared to the threat of Extreme Islamic Iran threatening to become a nuclear power, this is a much smaller threat to world peace, because no one is stopping Putin! Let&#39;s hope the world intervenes in Iran',
 'Reminder for my Canadian Friends Mr. Trudeau failed to tell Canadians that Canada receives a rebate every time Pfizer or Moderna vaccines are administered... Acuitas Therapeutics produces the lipid nanoparticle technology used in the Pfizer vaccine. The Trudeau Foundation is a shareholder. In total, she holds about 40 Acuitas.',
 'Professor Montagnier is the first to have spoken of the probable origin of a virus produced in the laboratory. This great man also warned about the dangers of mRNA. I am a Nobel Prize winner, I can work freely, there is no pressure on me RIP',
 'War in Ukraine According to spies, Putin is suffering from a brain disorder due to cancer treatment. Western spies believe Vladimir Putin&#39;s &quot;in

In [214]:
def find_type(label, pred_label):
    if label == 1 and pred_label == 1:
        return "TP"
    elif label == 1 and pred_label == 0:
        return "FN"
    elif label == 0 and pred_label == 1:
        return "FP"
    elif label == 0 and pred_label == 0:
        return "TN"

tmp["test_strat"] = tmp.apply(lambda row: find_type(row.label, row.test_misinfo), axis=1)
tmp["staging_strat"] = tmp.apply(lambda row: find_type(row.label, row.staging_misinfo), axis=1)
tmp.head()

,text,test_rating,language,translated_text,test_json,staging_rating,staging_json,test_misinfo,staging_misinfo,label,test_type,staging_type,test_strat,staging_strat
0,"#PriyankaOnABP has stolen the show. But, she h...",1,en,"#PriyankaOnABP has stolen the show. But, she h...","{'id': 'cb0522ddf36245c3905777a51ec42603', 'ra...",1,"{'id': '3efc1728d5124249938518d778998e05', 'ra...",0,0,0,TN,TN,TN,TN
1,Les informations que des troupes de l'ONU sont...,1,fr,Reports that UN troops are on Canadian soil an...,"{'id': '95167e50e1e34f2bb018244b1e09f87a', 'ra...",1,"{'id': '4a7e95ad0767483c8d8ea328eb0968a2', 'ra...",0,0,1,FN,FN,FN,FN
2,"Athletes Dropping Dead of Cardiac Arrest for ""...",5,en,"Athletes Dropping Dead of Cardiac Arrest for ""...","{'id': '5bfcd39db04a4f81a87b77f63e70293e', 'ra...",5,"{'id': 'aa4702cd58bc43caad4099a7ce374be3', 'ra...",1,1,1,TP,TP,TP,TP
3,I’ve been watching the local news and keeping ...,1,en,Ive been watching the local news and keeping u...,"{'id': '20f9f67f50764bd59a9d64168494fdec', 'ra...",1,"{'id': '4346ac82ede247599b28eb960e418ee7', 'ra...",0,0,0,TN,TN,TN,TN
4,"« 700.000 €, pour une personne qui assure la c...",1,fr,"700,000 for a person who provides operational ...","{'id': '8be0e7e42c1e40e29383119904753df1', 'ra...",1,"{'id': 'b3f75c1f9ecb4d57b8759c97d50e1b8d', 'ra...",0,0,0,TN,TN,TN,TN


In [222]:
tmp[tmp.test_type!=tmp.staging_type][["text", "staging_strat", "test_strat", "language"]].sort_values(["staging_strat", "test_strat"])

,text,staging_strat,test_strat,language
27,Rappel pour mes Amis Canadiens : M. Trudeau a ...,FN,TP,fr
91,Le Pr Montagnier est le premier à avoir parlé ...,FN,TP,fr
97,"Guerre en Ukraine : d'après les espions, Pouti...",FN,TP,fr
98,"STOP OU ENCORE ?\n\nLe passe sanitaire, nous d...",FN,TP,fr
210,Erbeutete Geheimdokumente: Ukraine forschte an...,FN,TP,de
243,VINDT U DIT NOG STEEDS OKAY ??????\n\nDE UPDAT...,FN,TP,nl
249,OTRA VIDA ROTA TRAS LA VACUNA\n\nEs peor el re...,FN,TP,es
20,https://lnkd.in/gmV3A4Pn\n\nDe geschiedenis he...,TN,FP,nl
184,Kritisch artikel over hoe de CDC omgegaan is m...,TP,FN,nl


In [229]:
print(json.dumps([{'post': row.text, 'token': '114d301f-2a5d-4bc6-9acd-57ba82de3ecf', 'expected_labels': ['MISINFORMATION']} for i, row in tmp[tmp.test_type!=tmp.staging_type].iterrows()], indent=4))

[
    {
        "post": "https://lnkd.in/gmV3A4Pn\n\nDe geschiedenis herhaalt zich.\nMaar vergeleken met t gevaar van het Extreem Islamitische Iran die dreigen een kernmacht te worden, is dit een veel kleiner gevaar voor de wereld vrede, want niemand stopt Putin!\nLaten we hopen dat de wereld wel ingrijpt in Iran !",
        "token": "114d301f-2a5d-4bc6-9acd-57ba82de3ecf",
        "expected_labels": [
            "MISINFORMATION"
        ]
    },
    {
        "post": "Rappel pour mes Amis Canadiens : M. Trudeau a omis de dire aux Canadiens que le Canada re\u00e7oit une ristourne chaque fois que des vaccins Pfizer ou Moderna sont administr\u00e9s...\n\nAcuitas Therapeutics produit la technologie des nanoparticules lipidiques utilis\u00e9e dans le vaccin Pfizer.\n\nLa Fondation Trudeau est un actionnaire. Au total, elle d\u00e9tient environ 40 % d'Acuitas.",
        "token": "114d301f-2a5d-4bc6-9acd-57ba82de3ecf",
        "expected_labels": [
            "MISINFORMATION"
        ]
    }

In [209]:
tmp[tmp.test_type==tmp.staging_type].language.value_counts()

en       197
fr        18
de        11
nl         6
es         4
pt         1
mg         1
bn         1
id         1
zh-CN      1
Name: language, dtype: int64

In [217]:
tmp.iloc[184]['test_json']['log_entry_json']

{'generatedRequestContext': {'classifierConfig': [{'classifierName': 'MISINFORMATION',
    'isEnabled': True},
   {'classifierName': 'CONTROVERSIAL'},
   {'classifierName': 'OBJECTIONABLE'}],
  'experimentFlagConfig': {'enableClassifierV2': True,
   'enableTranslationToEnglish': True}},
 'misinfoServiceLog': {'claimReviewSearchResponse': {}},
 'processedQuery': {'language': 'nl',
  'processedText': 'Kritisch artikel over hoe de CDC omgegaan is met de wetenschap rondom Covid-19',
  'rawText': 'Kritisch artikel over hoe de CDC omgegaan is met de wetenschap rondom Covid-19',
  'translatedText': 'Critical article on how the CDC has handled the science surrounding Covid-19'},
 'ratingRequest': {'account': {'id': 'johnsmith', 'name': 'John Smith'},
  'content': {'text': 'Kritisch artikel over hoe de CDC omgegaan is met de wetenschap rondom Covid-19'},
  'context': {'clientToken': '200b76c9-bc02-4397-a2d0-1440c9e1f6a0'}},
 'ratingResponse': {'id': '7f1a30a398c447d784ea0c02369e84a8',
  'severi

In [211]:
tmp.iloc[20]['staging_json']['log_entry_json']

{'generatedRequestContext': {'awsTraceId': '1-6263bb39-3c41e17b24cf59480e1e1730',
  'classifierConfig': [{'classifierName': 'MISINFORMATION', 'isEnabled': True},
   {'classifierName': 'CONTROVERSIAL'},
   {'classifierName': 'OBJECTIONABLE'}],
  'experimentFlagConfig': {'enableClassifierV2': True}},
 'misinfoServiceLog': {'claimReviewSearchResponse': {}},
 'ratingRequest': {'account': {'id': 'johnsmith', 'name': 'John Smith'},
  'content': {'text': 'https://lnkd.in/gmV3A4Pn\n\nDe geschiedenis herhaalt zich.\nMaar vergeleken met t gevaar van het Extreem Islamitische Iran die dreigen een kernmacht te worden, is dit een veel kleiner gevaar voor de wereld vrede, want niemand stopt Putin!\nLaten we hopen dat de wereld wel ingrijpt in Iran !'},
  'context': {'clientToken': '200b76c9-bc02-4397-a2d0-1440c9e1f6a0',
   'requestId': '1-6263bb39-3c41e17b24cf59480e1e1730'}},
 'ratingResponse': {'id': '234bbf586abb4963928c4419f7cd5426',
  'severity': 1,
  'version': '1'}}

In [91]:
for text in tmp[tmp.test_rating!=tmp.staging_rating].text:
    print(text, "\n")

Arizona: Republican Senate to Audit Maricopa County for 2020 Election Fraud  https://lnkd.in/dBuVrwr 

To crack the second wave of Corona, it will have to be strict lockdown like last year: AIIMS Director Randeep Guleria 

While we await official final report, a leaked draft of joint report from World Health Organization (WHO) and China, first reported by Associated Press, says it’s “extremely unlikely” that coronavirus escaped from lab, and animal transmission of COVID was most likely.  But New York Post reports, “both Republicans and Democrats are slamming the report as biased because of the Chinese Communist Party-led government’s role in the investigation after party officials covered up early case data before the virus spread abroad.”  As has been widely reported, entire WHO COVID origins investigation was fraudulent and biased from start to finish.   It was, say experts, essentially a Chinese Communist Party (CCP) conducted, sham inquiry, handed over to, and rubber stamped by, WH

In [103]:
tmp[tmp.test_rating!=tmp.staging_rating]

,text,test_rating,test_json,staging_rating,staging_json
7,Arizona: Republican Senate to Audit Maricopa C...,1,<Response [200]>,3,<Response [200]>
8,"To crack the second wave of Corona, it will ha...",3,<Response [200]>,4,<Response [200]>
48,"While we await official final report, a leaked...",1,<Response [200]>,4,<Response [200]>
70,WE HAVE HOPE FOR THE END OF COVID. Ivermectin...,4,<Response [200]>,5,<Response [200]>
85,ANSYS FLUENT Training : Corona Virus Dispersio...,1,<Response [200]>,3,<Response [200]>
94,Laptop evidence ‘strongly indicates’ Joe Biden...,4,<Response [200]>,5,<Response [200]>


In [119]:
tmp.iloc[48].text

'While we await official final report, a leaked draft of joint report from World Health Organization (WHO) and China, first reported by Associated Press, says it’s “extremely unlikely” that coronavirus escaped from lab, and animal transmission of COVID was most likely.  But New York Post reports, “both Republicans and Democrats are slamming the report as biased because of the Chinese Communist Party-led government’s role in the investigation after party officials covered up early case data before the virus spread abroad.”  As has been widely reported, entire WHO COVID origins investigation was fraudulent and biased from start to finish.   It was, say experts, essentially a Chinese Communist Party (CCP) conducted, sham inquiry, handed over to, and rubber stamped by, WHO.  Jamie Metzl, former staffer to President Biden when he led Senate Foreign Affairs Committee, and formerly on President Bill Clinton National Security Council told CBS that it’s as if we had Soviet Union investigate Che

In [109]:
"WE HAVE HOPE FOR THE END OF COVID. Ivermectin can kill COVID-19 within 48 hours, Monash University study finds"

'WE HAVE HOPE FOR THE END OF COVID. Ivermectin can kill COVID-19 within 48 hours, Monash University study finds'

In [106]:
"To crack the second wave of Corona, it will have to be strict lockdown like last year: AIIMS Director Randeep Guleria"

'To crack the second wave of Corona, it will have to be strict lockdown like last year: AIIMS Director Randeep Guleria'

In [120]:
tmp[tmp.test_rating!=tmp.staging_rating].to_csv("tmp.csv", index=False)

In [92]:
tmp.iloc[7].text

'Arizona: Republican Senate to Audit Maricopa County for 2020 Election Fraud  https://lnkd.in/dBuVrwr'

In [81]:
for i, row in tmp[tmp.test_rating!=tmp.staging_rating].iterrows():
    print(len(row.test_json.json()['log_entry_json']['misinfoServiceLog']['claimReviewSearchResponse']),
          len(row.staging_json.json()['log_entry_json']['misinfoServiceLog']['claimReviewSearchResponse']))

0 1
1 1
0 1
1 1
0 1
1 1


In [84]:
tmp.iloc[8].test_json.json()

{'id': 'bcc4bd71b4144f959bf7a265a20d1824',
 'rating': {'severity': 3,
  'version': '1',
  'categories': [{'details': 'Similar to claim "De coronapandemie is gepland." (https://dpa-factchecking.com/belgium/211014-99-595955)\nSimilar to claim "Erst der Lockdown light, dann der harte Lockdown - über die aktuellen Maßnahmen zur Bekämpfung der Corona-Pandemie wird viel diskutiert. In Beiträgen in sozialen Netzwerken vom Dezember wird der Virologe Hendrik Streeck mit der Forderung nach einer Rückkehr zur Lebensnormalität zitiert (hier archiviert). Es werde nicht einmal durch den härtesten Lockdown gelingen, das Virus zu besiegen, soll er gesagt haben. «Endlich redet er klartext» (sic!), kommentieren Nutzer dies." (https://dpa-factchecking.com/germany/201217-99-727315)\nSimilar to claim "The Government of India has decided to impose a complete lockdown from 29 Feb to 31 Feb 2021" (https://factly.in/satirical-message-about-government-imposing-lockdown-from-29-feb-to-31-feb-2021-is-shared-as-re

In [89]:
tmp.iloc[8].staging_json.json()

{'id': '92f45edb61184b64b873352080049ebc',
 'rating': {'severity': 4,
  'version': '1',
  'categories': [{'details': 'Similar to claim "De coronapandemie is gepland." (https://dpa-factchecking.com/belgium/211014-99-595955)\nSimilar to claim "Erst der Lockdown light, dann der harte Lockdown - über die aktuellen Maßnahmen zur Bekämpfung der Corona-Pandemie wird viel diskutiert. In Beiträgen in sozialen Netzwerken vom Dezember wird der Virologe Hendrik Streeck mit der Forderung nach einer Rückkehr zur Lebensnormalität zitiert (hier archiviert). Es werde nicht einmal durch den härtesten Lockdown gelingen, das Virus zu besiegen, soll er gesagt haben. «Endlich redet er klartext» (sic!), kommentieren Nutzer dies." (https://dpa-factchecking.com/germany/201217-99-727315)\nSimilar to claim "The Government of India has decided to impose a complete lockdown from 29 Feb to 31 Feb 2021" (https://factly.in/satirical-message-about-government-imposing-lockdown-from-29-feb-to-31-feb-2021-is-shared-as-re

In [110]:
d = pd.read_csv("logs-insights-results (47).csv")
d

,@timestamp,kubernetes.container_name,severity,text
0,2022-04-22 17:13:41.261,claimreview-searcher-service-container,NaN,Arizona: Republican Senate to Audit Maricopa C...
1,2022-04-22 17:13:42.196,claimreview-searcher-service-container,NaN,"To crack the second wave of Corona, it will ha..."
2,2022-04-22 17:13:49.168,claimreview-searcher-service-container,NaN,"While we await official final report, a leaked..."
3,2022-04-22 17:13:51.244,claimreview-searcher-service-container,NaN,WE HAVE HOPE FOR THE END OF COVID. Ivermectin ...
4,2022-04-22 17:13:52.862,claimreview-searcher-service-container,NaN,ANSYS FLUENT Training Corona Virus Dispersion ...
5,2022-04-22 17:13:53.692,claimreview-searcher-service-container,NaN,Laptop evidence strongly indicates Joe Biden i...
6,2022-04-22 17:19:25.814,claimreview-searcher-service-container,NaN,Arizona: Republican Senate to Audit Maricopa C...
7,2022-04-22 17:19:26.786,claimreview-searcher-service-container,NaN,"To crack the second wave of Corona, it will ha..."
8,2022-04-22 17:19:33.963,claimreview-searcher-service-container,NaN,"While we await official final report, a leaked..."
9,2022-04-22 17:19:36.223,claimreview-searcher-service-container,NaN,WE HAVE HOPE FOR THE END OF COVID. Ivermectin ...


In [111]:
d.text.value_counts()

Arizona: Republican Senate to Audit Maricopa County for 2020 Election Fraud https://lnkd.in/dBuVrwr                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     